In [1]:
import psycopg2 as pg
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def db_select(query):
    conn = None
    cur = None
    results = []
    try:
        conn = pg.connect("dbname=overflow user=postgres password=postgres")
        cur = conn.cursor()
        cur.execute(query)
        results = cur.fetchall()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
        if cur is not None:
            cur.close()
        return results

In [3]:
def db_write(query):
    conn = None
    cur = None
    results = []
    try:
        conn = pg.connect("dbname=overflow user=postgres password=postgres")
        cur = conn.cursor()
        cur.execute(query)
        conn.commit()
        results = cur.fetchone()
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
        if cur is not None:
            cur.close()
        return

In [4]:
create_statement = """
CREATE TABLE badges (
	id SERIAL PRIMARY KEY,
	user_id INTEGER NOT NULL,
	class SMALLINT NOT NULL,
	name VARCHAR(256) NOT NULL,
	tag_based BOOLEAN NOT NULL,
	date TIMESTAMP NOT NULL
);

CREATE TABLE comments (
	id SERIAL PRIMARY KEY,
	post_id INTEGER NOT NULL,
	user_id INTEGER,
	score SMALLINT DEFAULT 0,
	content_license VARCHAR(64) NOT NULL,
	user_display_name VARCHAR(256),
	text TEXT,
	creation_date TIMESTAMP NOT NULL
);

CREATE TABLE history (
	id SERIAL PRIMARY KEY,
	post_id INTEGER NOT NULL,
	user_id INTEGER,
	post_history_type_id SMALLINT NOT NULL,
	user_display_name VARCHAR(256),
	content_license VARCHAR(64),
	revision_guid uuid,
	text TEXT,
	comment TEXT,
	creation_date TIMESTAMP NOT NULL
);

CREATE TABLE links (
	id SERIAL PRIMARY KEY,
	related_post_id INTEGER NOT NULL,
	post_id INTEGER NOT NULL,
	link_type_id SMALLINT NOT NULL,
	creation_date TIMESTAMP NOT NULL
);

CREATE TABLE posts (
	id SERIAL PRIMARY KEY,
	owner_user_id INTEGER,
	last_editor_user_id INTEGER,
	post_type_id SMALLINT NOT NULL,
	accepted_answer_id INTEGER,
	score INTEGER DEFAULT 0,
	parent_id INTEGER,
	view_count INTEGER,
	answer_count INTEGER DEFAULT 0,
	comment_count INTEGER DEFAULT 0,
	owner_display_name VARCHAR(256),
	last_editor_display_name VARCHAR(256),
	title VARCHAR(512),
	tags VARCHAR(512),
	content_license VARCHAR(64) NOT NULL,
	body TEXT,
	favorite_count INTEGER,
	creation_date TIMESTAMP NOT NULL,
	community_owned_date TIMESTAMP,
	closed_date TIMESTAMP,
	last_edit_date TIMESTAMP,
	last_activity_date TIMESTAMP
);

CREATE TABLE tags (
	id SERIAL PRIMARY KEY,
	excerpt_post_id INTEGER,
	wiki_post_id INTEGER,
	tag_name VARCHAR(256) NOT NULL,
	count INTEGER DEFAULT 0
);

CREATE TABLE users (
	id SERIAL PRIMARY KEY,
	account_id INTEGER,
	reputation INTEGER NOT NULL DEFAULT 0,
	views INTEGER DEFAULT 0,
	down_votes INTEGER DEFAULT 0,
	up_votes INTEGER DEFAULT 0,
	display_name VARCHAR(256) NOT NULL,
	location VARCHAR(512),
	profile_image_url VARCHAR(256),
	website_url VARCHAR(256),
	about_me TEXT,
	creation_date TIMESTAMP NOT NULL,
	last_access_date TIMESTAMP NOT NULL
);

CREATE TABLE votes (
	id SERIAL PRIMARY KEY,
	user_id INTEGER,
	post_id INTEGER NOT NULL,
	vote_type_id SMALLINT NOT NULL,
	bounty_amount SMALLINT DEFAULT 0,
	creation_date TIMESTAMP NOT NULL
);
"""
db_write(create_statement);

In [5]:
filenames = ['Badges', 'Comments', 'PostHistory', 'PostLinks', 'Posts', 'Tags', 'Users', 'Votes']
data = {}
for name in filenames:
    with open(f'./data/{name}.xml', encoding="utf8") as f:
        soup = BeautifulSoup(f.read(), 'lxml')
        data[name] = soup.find_all('row')